In [1]:
import numpy as np
import zebende as zb
import pandas as pd
from numba import cuda
import cupy as cp

In [2]:
tws = pd.read_csv('./test_data/time_window_scales.txt', header=None).to_numpy().flatten()[:]
tws_D = cuda.to_device(tws)

In [3]:
data_1 = pd.read_csv('./test_data/data_pre_proc/S001/S001_03/S001_03_F3.txt', header=None)
data_2 = pd.read_csv('./test_data/data_pre_proc/S001/S001_03/S001_03_F6.txt', header=None)
data_3 = pd.read_csv('./test_data/data_pre_proc/S001/S001_03/S001_03_P3.txt', header=None)
data_4 = pd.read_csv('./test_data/data_pre_proc/S001/S001_03/S001_03_P6.txt', header=None)

In [4]:
data = pd.concat([data_1, data_2, data_3, data_4], axis=1).to_numpy(dtype=np.float64)
data . data.shape

(15742, 4)

In [5]:
int_data = zb.integrated_series(data).T
int_data_D = cp.array(int_data)

In [6]:
DCCA_of = zb.mat_index_comb(int_data, axis=0)
DCCA_of_D = cp.array(DCCA_of)

In [7]:
time_steps = cp.arange(data.shape[0])

# outputs
F_DFA_arr = cp.zeros(shape=(tws.shape[0], data.shape[1], tws.shape[0]), dtype=data.dtype)
DCCA_arr = cp.zeros(shape=(tws.shape[0], DCCA_of.shape[0], tws.shape[0]), dtype=data.dtype)
P_DCCA_arr = cp.ones(shape=(DCCA_of.max() + 1, DCCA_of.max() + 1, tws.shape[0]), dtype=data.dtype)

# auxiliary arrays
detrend = cp.zeros(shape=(data.shape[0] - tws[0], data.shape[1], tws.shape[0]), dtype=data.dtype)

f2dfa_n = cp.zeros(shape=(data.shape[0] - tws[0], data.shape[1], tws.shape[0]), dtype=data.dtype)

dcca_n = cp.zeros(shape=(data.shape[0] - tws[0], DCCA_of.shape[0], tws.shape[0]), dtype=data.dtype)


In [8]:
int_data_D.shape

(4, 15742)

In [9]:
@cuda.jit()
def cuda_f2dfa(data, tws, time_steps, # input
                f2dfa_n  # Output
                ):
    
    x,y,z= cuda.grid(3)

    # if (x <4) and (y <4) and (z<4):
    #     ...

    if (x < data.shape[0] )  and (y < data.shape[1] -tws[z]) and (z < tws[z]):
        x_sum = 0
        y_sum = 0
        xy_sum = 0
        x2_sum = 0
        n_pt_w = tws[z] + 1

        # fit line
        for i in range(n_pt_w):
            x_sum += time_steps[x + i]
            y_sum += data[x, y + i]
            xy_sum += time_steps[x + i] * data[x, y + i]
            x2_sum += time_steps[x + i]**2
        slope = ( ((n_pt_w * xy_sum) - (x_sum * y_sum)) / ((n_pt_w * x2_sum) - (x_sum**2)) )
        inter = ( (y_sum - (slope * x_sum)) / (n_pt_w) )

        # detrended mean
        tmp = 0
        for i in range(n_pt_w):
            tmp += ((data[x, y + i] - (slope * time_steps[x + i] + inter))**2)/n_pt_w

        # return value
        f2dfa_n[x,y,z] = tmp
    

In [10]:
GridDimX = data.shape[0]
GridDimY = data.shape[1]
GridDimZ = tws.shape[0]

GridDimX, GridDimY, GridDimZ

(15742, 4, 42)

In [11]:

tpb = (int(np.ceil(GridDimX/64)), int(np.ceil(GridDimY/64)), int(np.ceil(GridDimZ/64)))
bpg = (64,64, 64)

tpb , bpg

((246, 1, 1), (64, 64, 64))

In [26]:
cuda_f2dfa[bpg, tpb](int_data_D, tws_D, time_steps, f2dfa_n)
DFA_gpu = cp.sqrt(f2dfa_n.sum(axis=0))

In [27]:
type(f2dfa_n), f2dfa_n.device

(cupy.ndarray, <CUDA Device 0>)

In [28]:
DFA_cpu = cp.asnumpy(DFA_gpu)
DFA_cpu.shape

(4, 42)

In [25]:
DFA_cpu.T

array([[   8.84194549,   10.63860893,   16.10279479, ...,    0.        ,
           0.        ,    0.        ],
       [  10.43072385,   16.23400312,   19.2122646 , ...,    0.        ,
           0.        ,    0.        ],
       [  13.6057914 ,   17.26211617,   16.75843782, ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [4879.86050465, 5515.31359409, 5419.30412669, ...,    0.        ,
           0.        ,    0.        ],
       [4690.3891318 , 6257.71574151, 6260.40679493, ...,    0.        ,
           0.        ,    0.        ],
       [4849.32439126, 6409.69878436, 7316.17314371, ...,    0.        ,
           0.        ,    0.        ]])

In [21]:
np.unique(DFA_cpu).size

168